In [1]:
import requests
import pandas as pd
import time
#from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np
import os
import re

Find relevant season ID's

In [30]:
seasons_info = pd.read_csv('data/seasons_info.csv', index_col=0)

In [31]:
index_rows = []
for year in ['2019', '2020', '2021', '2022', '2023']:
    frames = [index for index, row in seasons_info.iterrows() if row['starting_at'].split('-')[0]==year]
    index_rows = index_rows + frames
index_rows

relevant_seasons = seasons_info.iloc[index_rows,:]

In [32]:
relevant_seasons.to_csv('data/analyse/relevant_seasons.csv', index=False)

Count types of events for seasons/leagues

In [19]:
eventType_dict = {'typeId':[], 'count':[], 'season':[]}

In [ ]:
id_list = [i for i in relevant_seasons['id']]
path = 'data/events'
for file in os.listdir(path): #loops over event folder
    if int(file.split('_')[1].strip('.csv')) in id_list: #checks is relevant season is in file title
        event_file = pd.read_csv(f'{path}/{file}') #reads file
        for index, value in event_file['type_id'].value_counts().items(): #loops and appends all events to dict
            eventType_dict['typeId'].append(index)
            eventType_dict['count'].append(value)
            eventType_dict['season'].append(int(file.split('_')[1].strip('.csv')))
#saving file
eventType_dict = pd.DataFrame.from_dict(eventType_dict)
eventType_dict.to_csv('data/analyse/allEvents.csv', index=False)

Creating full var file

In [7]:
var_events = pd.read_csv('data/var.csv')
path = 'data/events'
for file in os.listdir(path):
    event_file = pd.read_csv(f'{path}/{file}')
    event_file = event_file[event_file['type_id']==10]
    var_events = pd.concat([var_events, event_file])

var_events.to_csv('data/var.csv')

/tmp/ipykernel_3224/4196393108.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  var_events = pd.concat([var_events, event_file])


In [8]:
var_events

,id,type_id,sub_type_id,fixture_id,player_id,related_player_id,period_id,participant_id,addition,minute,sort_order,Unnamed: 0
1055,86885139,10,NaN,11344409,827.0,NaN,3239207,43,NaN,16,1,1055
1056,86885147,10,NaN,11344409,132577.0,NaN,435544,43,NaN,48,1,1056
75,88033873,10,NaN,11867440,53.0,NaN,3370695,13,NaN,5,1,75
77,88033881,10,NaN,11867440,5141.0,NaN,3370696,13,Goal Disallowed,78,1,77
81,88033803,10,NaN,11867447,61780.0,NaN,3370710,78,NaN,47,1,81
...,...,...,...,...,...,...,...,...,...,...,...,...
3043,95003466,10,NaN,18867501,37347355.0,NaN,5194480,4070,Goal Disallowed,45,1,3043
3056,95216799,10,NaN,18867570,133957.0,NaN,5199492,613,Goal Disallowed,90,1,3056
3058,95807098,10,NaN,18867516,130103.0,NaN,5217388,585,Penalty Cancelled,45,1,3058
3071,107500312,10,NaN,18867530,49420.0,NaN,5248342,585,Goal Disallowed,32,1,3071


Adding season name to full event file

In [22]:
seasons = pd.read_csv('data/seasons.csv', index_col=0)
eventType_dict = eventType_dict.merge(seasons[["id", "league_id", "name"]], left_on='season', right_on='id')
eventType_dict 

,typeId,count,season,id,league_id,name
0,19,1818,17361,17361,82,2020/2021
1,10,140,17361,17361,82,2020/2021
2,21,30,17361,17361,82,2020/2021
3,20,22,17361,17361,82,2020/2021
4,19,1965,17420,17420,8,2020/2021
...,...,...,...,...,...,...
59,20,38,21795,21795,82,2023/2024
60,19,2690,21818,21818,384,2023/2024
61,10,258,21818,21818,384,2023/2024
62,20,78,21818,21818,384,2023/2024


In [23]:
eventType_dict.to_csv('data/analyse/allEvents.csv', index=False)

Creating event type dataset for all leagues

In [2]:
eventType_dict = pd.read_csv('data/analyse//allEvents.csv')
types = pd.read_csv('data/types.csv')

In [3]:
#get all events for each league
for i in eventType_dict['league_id'].unique():
    events = eventType_dict.loc[eventType_dict['league_id']==i]
    league = events[['typeId', 'count']].groupby('typeId').sum()
    league = league.merge(types[["id", "name"]], left_on='typeId', right_on='id')
    league.to_csv(f'data/analyse/countPerLeague/league{i}.csv')

In [3]:
import plotly.graph_objects as go
import plotly
import os
import pandas as pd

path = 'data/analyse/countPerLeague'
size=[]
y=[]
league_names = []

for file in os.listdir(path):
    event_file = pd.read_csv(f'{path}/{file}')
    type_names = [i for i in event_file['name']]
    event_file = pd.read_csv(f'{path}/{file}')
    size = size + [i for i in event_file['count']]
    y = y + [i for i in event_file['Unnamed: 0']]
    league_names.append(file.split('.')[0])
    print(event_file)

   Unnamed: 0  count  id             name
0           0    966  10              VAR
1           1  12158  19       Yellowcard
2           2    374  20          Redcard
3           3    252  21  Yellow/Red card
   Unnamed: 0  count  id             name
0           0    837  10              VAR
1           1  11755  19       Yellowcard
2           2    457  20          Redcard
3           3    293  21  Yellow/Red card
   Unnamed: 0  count  id             name
0           0   1184  10              VAR
1           1   8966  19       Yellowcard
2           2    177  20          Redcard
3           3    122  21  Yellow/Red card
   Unnamed: 0  count  id             name
0           0    686  10              VAR
1           1   6902  19       Yellowcard
2           2    104  20          Redcard
3           3    132  21  Yellow/Red card


In [4]:
size = size
fig = go.Figure(data=[go.Scatter(
    x=[1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4],
    y=y,
    mode='markers',
    marker=dict(
        size=size,
        sizemode='area',
        #sizeref=2.*max(size)/(40.**2),
        color=size,
        sizemin=4
    )
)])

fig.show()

In [5]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

y = size

color = ['rgb(93, 164, 214)', 'rgb(93, 164, 214)','rgb(93, 164, 214)','rgb(93, 164, 214)', 'rgb(255, 144, 14)','rgb(255, 144, 14)','rgb(255, 144, 14)','rgb(255, 144, 14)', 'rgb(44, 160, 101)','rgb(44, 160, 101)','rgb(44, 160, 101)','rgb(44, 160, 101)', 'rgb(255, 65, 54)','rgb(255, 65, 54)','rgb(255, 65, 54)','rgb(255, 65, 54)']
#cut_interval = [600, 600]
cut_interval = [1500, 6500]
bar = go.Bar(
    y=y,
    name='foo',
    marker_color=color,
)
fig = make_subplots(
    rows=2,
    cols=1,
    vertical_spacing=0.05,
    shared_xaxes=True,
)

fig.append_trace(bar, row=1, col=1)
bar.showlegend = False
fig.append_trace(bar, row=2, col=1)

fig.update_yaxes(range=[cut_interval[1], max(y) * 1.1], row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_yaxes(range=[0, cut_interval[0]], row=2, col=1)


With log()

In [ ]:
import math   
path = 'data/analyse/countPerLeague'
size2=[]
y=[]

for file in os.listdir(path):
    x_axis = 1
    event_file = pd.read_csv(f'{path}/{file}')
    size2 = size2 + [math.log(i) for i in event_file['count']]
    y = y + [i for i in event_file['Unnamed: 0']]
    print(size2)

[966, 12158, 374, 252, 837, 11755, 457, 293, 1184, 8966, 177, 122, 686, 6902, 104, 132, 966, 12158, 374, 252, 837, 11755, 457, 293, 1184, 8966, 177, 122, 686, 6902, 104, 132, 6.873163834212518, 9.405742668308667, 5.924255797414532, 5.529429087511423, 6.729824070489475, 9.37203396097417, 6.124683390894205, 5.680172609017068, 7.076653815443951, 9.101194924715385, 5.176149732573829, 4.804021044733257, 6.530877627725885, 8.839566503657949, 4.6443908991413725, 4.882801922586371, 6.873163834212518, 9.405742668308667, 5.924255797414532, 5.529429087511423, 6.729824070489475, 9.37203396097417, 6.124683390894205, 5.680172609017068, 7.076653815443951, 9.101194924715385, 5.176149732573829, 4.804021044733257, 6.530877627725885, 8.839566503657949, 4.6443908991413725, 4.882801922586371]
[966, 12158, 374, 252, 837, 11755, 457, 293, 1184, 8966, 177, 122, 686, 6902, 104, 132, 966, 12158, 374, 252, 837, 11755, 457, 293, 1184, 8966, 177, 122, 686, 6902, 104, 132, 6.873163834212518, 9.405742668308667, 5.92

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

y = size2

color = ['rgb(93, 164, 214)', 'rgb(93, 164, 214)','rgb(93, 164, 214)','rgb(93, 164, 214)', 'rgb(255, 144, 14)','rgb(255, 144, 14)','rgb(255, 144, 14)','rgb(255, 144, 14)', 'rgb(44, 160, 101)','rgb(44, 160, 101)','rgb(44, 160, 101)','rgb(44, 160, 101)', 'rgb(255, 65, 54)','rgb(255, 65, 54)','rgb(255, 65, 54)','rgb(255, 65, 54)']

fig = px.bar(x=range(16), y=y, color=color)
fig.show()

In [10]:
cut_interval = [1500, 6500]

data=[
    go.Bar(name='VAR', x=league_names, y=[966, 837, 1184,686], marker_color='blue', showlegend=False),
    go.Bar(name='Yellowcard', x=league_names, y=[12158, 11755, 8966, 6902], marker_color='red', showlegend=False),
    go.Bar(name='Redcard', x=league_names, y=[374, 457, 177, 104], marker_color='yellow', showlegend=False),
    go.Bar(name='Yellow/Red card', x=league_names, y=[252, 293, 122, 132], marker_color='orange', showlegend=False)
]

fig = make_subplots(
    rows=2,
    cols=1,
    vertical_spacing=0.05,
    shared_xaxes=True,
)
for trace in data:
    fig.add_trace(trace, row=1, col=1)

# Adjust y-axis range for the upper part
fig.update_yaxes(range=[cut_interval[1], max(max(trace.y for trace in data)) * 1.1], row=1, col=1)
fig.update_xaxes(visible=False, row=1, col=1)

# Add data to the second subplot (lower part) with low range
for trace in data:
    fig.add_trace(trace, row=2, col=1)

# Adjust y-axis range for the lower part
fig.update_yaxes(range=[0, cut_interval[0]], row=2, col=1)

fig.show()

In [80]:
id_list = [i for i in relevant_seasons['id']]
fixtures_dict = {'fixture_id':[], 'league_id':[], 'referee_id':[], 'season_id':[], 'game_name':[]}
path = 'data/fixtures'
for file in os.listdir(path):
    if int(file.split('_')[1].strip('.csv')) in id_list:
        event_file = pd.read_csv(f'{path}/{file}')
        fixtures_dict['fixture_id'] = fixtures_dict['fixture_id']  + [i for i in event_file['fixture_id']]
        fixtures_dict['league_id'] = fixtures_dict['league_id']  + [i for i in event_file['league_id']]
        fixtures_dict['referee_id'] = fixtures_dict['referee_id']  + [i for i in event_file['referee']]
        fixtures_dict['season_id'] = fixtures_dict['season_id']  + [i for i in event_file['season_id']]
        fixtures_dict['game_name'] = fixtures_dict['game_name']  + [i for i in event_file['name']]
fixtures_dict = pd.DataFrame.from_dict(fixtures_dict)
fixtures_dict

,fixture_id,league_id,referee_id,season_id,game_name
0,11867232,8,15293,16036,Watford vs Brighton & Hove Albion
1,11867238,8,14532,16036,Brighton & Hove Albion vs West Ham United
2,11867248,8,15242,16036,Brighton & Hove Albion vs Southampton
3,11953357,27,19850,16216,Bristol Rovers vs Brighton & Hove Albion
4,11867262,8,13533,16036,Manchester City vs Brighton & Hove Albion
...,...,...,...,...,...
6031,18867540,384,14082,21818,Empoli vs Cagliari
6032,18867403,384,13984,21818,Juventus vs Hellas Verona
6033,18867472,384,16780,21818,Hellas Verona vs Cagliari
6034,18867582,384,15306,21818,Hellas Verona vs Juventus


In [4]:
var_events = pd.read_csv('var.csv')
path = 'data/events'
for file in os.listdir(path):
    event_file = pd.read_csv(f'{path}/{file}')
    event_file = event_file[event_file['type_id']==10]
    var_events = pd.concat([var_events, event_file])

var_events.to_csv('var.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'var.csv'

In [69]:
teams = pd.read_csv('data/teams.csv')
teams = teams[['id', 'name']].drop_duplicates()
teams.to_csv('teams_updates.csv', index=False)
teams = teams.rename(columns={'id': 'team_id','venue_id': 'team_venue_id', 'name': 'team_name'})
teams

,team_id,team_name
0,78,Brighton & Hove Albion
1,11,Fulham
2,8,Liverpool
3,19,Arsenal
4,42,Leicester City
...,...,...
840,3565,Eintracht Braunschweig
856,9133,Catania
859,3691,Siena
879,10714,Livorno


In [84]:
var_events = pd.read_csv('var.csv')
var_events = var_events[['id','addition', 'fixture_id','participant_id', 'minute']]
var_events = var_events.rename(columns={'id': 'event_id'})
var_events

,event_id,addition,fixture_id,participant_id,minute
0,86885139,NaN,11344409,43,16
1,86885147,NaN,11344409,43,48
2,88033873,NaN,11867440,13,5
3,88033881,Goal Disallowed,11867440,13,78
4,88033803,NaN,11867447,78,47
...,...,...,...,...,...
4146,95003466,Goal Disallowed,18867501,4070,45
4147,95216799,Goal Disallowed,18867570,613,90
4148,95807098,Penalty Cancelled,18867516,585,45
4149,107500312,Goal Disallowed,18867530,585,32


In [85]:
var_per_referee = var_events.merge(teams, left_on='participant_id', right_on='team_id')
var_per_referee

,event_id,addition,fixture_id,participant_id,minute,team_id,team_name
0,86885139,NaN,11344409,43,16,43,Lazio
1,86885147,NaN,11344409,43,48,43,Lazio
2,88033873,NaN,11867440,13,5,13,Everton
3,88033881,Goal Disallowed,11867440,13,78,13,Everton
4,88033803,NaN,11867447,78,47,78,Brighton & Hove Albion
...,...,...,...,...,...,...,...
4137,95003466,Goal Disallowed,18867501,4070,45,4070,Frosinone
4138,95216799,Goal Disallowed,18867570,613,90,613,Torino
4139,95807098,Penalty Cancelled,18867516,585,45,585,Cagliari
4140,107500312,Goal Disallowed,18867530,585,32,585,Cagliari


In [86]:
var_per_referee = var_per_referee.merge(fixtures_dict, left_on='fixture_id', right_on='fixture_id')
var_per_referee.to_csv('data/bog_data.csv')
var_per_referee

,event_id,addition,fixture_id,participant_id,minute,team_id,team_name,league_id,referee_id,season_id,game_name
0,88033873,NaN,11867440,13,5,13,Everton,8,11691,16036,Everton vs Brighton & Hove Albion
1,88033881,Goal Disallowed,11867440,13,78,13,Everton,8,11691,16036,Everton vs Brighton & Hove Albion
2,88033803,NaN,11867447,78,47,78,Brighton & Hove Albion,8,74959,16036,Brighton & Hove Albion vs Aston Villa
3,88033815,NaN,11867447,78,90,78,Brighton & Hove Albion,8,74959,16036,Brighton & Hove Albion vs Aston Villa
4,88033765,Goal Confirmed,11867456,78,82,78,Brighton & Hove Albion,8,15242,16036,AFC Bournemouth vs Brighton & Hove Albion
...,...,...,...,...,...,...,...,...,...,...,...
4135,95003466,Goal Disallowed,18867501,4070,45,4070,Frosinone,384,20061,21818,Frosinone vs Cagliari
4136,95216799,Goal Disallowed,18867570,613,90,613,Torino,384,14031,21818,Cagliari vs Torino
4137,95807098,Penalty Cancelled,18867516,585,45,585,Cagliari,384,20345,21818,Roma vs Cagliari
4138,107500312,Goal Disallowed,18867530,585,32,585,Cagliari,384,17312,21818,Cagliari vs Napoli


## Save full csv for each league

Csv with all VAR events, type_id 10, including all info necessary.

In [89]:
leagues = [8,82,384,564]
for league in leagues:
    league_data = var_per_referee[var_per_referee['league_id']==league]
    league_data.to_csv(f'data/analyse/full_{league}.csv')

## Testing 

In [ ]:
#load data
full_8 = pd.read_csv('data/analyse/full_8.csv')


In [ ]:
for team in full_8['team_name'].unique():
    team_data = full_8[full_8['team_name']==team]
    team_df = {'referee_id':[], 'goal_ratio':[]}
    for referee in team_data['referee_id']:
        team_referee_data = team_data[team_data['referee_id']==referee]
        goal_disallowed = team_referee_data[team_referee_data['addition']=='Goal Disallowed'].size
        goal_confirmed = team_referee_data[team_referee_data['addition']=='Goal Confirmed'].size
        try:
            error_ratio = goal_confirmed/goal_disallowed
        except ZeroDivisionError:
            error_ratio = 0
        
        team_df['referee_id'].append(referee)
        team_df['goal_ratio'].append(error_ratio)
    team_df = pd.DataFrame.from_dict(team_df)
    team_df.to_csv(f'data/analyse/8/team_{team}.csv')

In [ ]:
for

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   z=[[1, None, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, -10, 20]],
                   x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                   y=['Morning', 'Afternoon', 'Evening'],
                   hoverongaps = False))
fig.show()